In [1]:
import pandas as pd

In [30]:
#Temperature data
GitURL_TempAdam = 'https://raw.githubusercontent.com/StevenPeutz/MDA_HeatWaveAnalysis/master/dataset/amsterdamdailytemperature.csv'
#GitURL_TempRdam = '..'
#GitURL_TempBru = '..'
#GitURL_TempAnt = '..'

#Mortality
GitURL_BelgiumDeath = 'https://raw.githubusercontent.com/StevenPeutz/MDA_HeatWaveAnalysis/master/dataset/cause_of_death.xlsx'
GitURL_NetherlandsDeath = 'https://raw.githubusercontent.com/StevenPeutz/MDA_HeatWaveAnalysis/master/dataset/mortalityNetherlands.csv'
GitURL_NetherlandsDeath2 = 'https://raw.githubusercontent.com/StevenPeutz/MDA_HeatWaveAnalysis/master/dataset/mortalityNL_manuallycleaned.csv'
GitURL_NetherlandsDeath3 = 'https://raw.githubusercontent.com/StevenPeutz/MDA_HeatWaveAnalysis/master/dataset/mortalityNL_manuallycleaned.xlsx'

In [31]:
df_MortalityBE = pd.read_excel(GitURL_BelgiumDeath)

In [34]:
df_MortalityBE.shape

(41278, 7)

In [69]:
df_MortalityBE.head(15)

,REGION,MONTH,YEAR,AGE-GROUP,SEX,COD,COUNT
0,2000,1,2009,0-24,F,Certain infectious and parasitic diseases,1
1,2000,1,2009,0-24,F,Neoplasms,4
2,2000,1,2009,0-24,F,"Endocrine, nutritional and metabolic diseases",1
3,2000,1,2009,0-24,F,Diseases of the nervous system,1
4,2000,1,2009,0-24,F,Diseases of the respiratory system,1
5,2000,1,2009,0-24,F,Certain conditions originating in the perinata...,7
6,2000,1,2009,0-24,F,"Congenital malformations, deformations and chr...",5
7,2000,1,2009,0-24,F,"Symptoms, signs and abnormal clinical and labo...",3
8,2000,1,2009,0-24,F,External causes of morbidity and mortality,3
9,2000,1,2009,0-24,M,Certain infectious and parasitic diseases,1


In [36]:
df_MortalityBE['COD'].value_counts()

External causes of morbidity and mortality                                                             3768
Neoplasms                                                                                              3628
Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified                3536
Diseases of the circulatory system                                                                     3380
Diseases of the nervous system                                                                         3196
Diseases of the digestive system                                                                       3075
Diseases of the respiratory system                                                                     3037
Endocrine, nutritional and metabolic diseases                                                          2902
Certain infectious and parasitic diseases                                                              2879
Mental and behavioural disor

In [37]:
#first make it easier by creating a df with only the relevant rows (Belgium dataset)
values_to_keep = ['Neoplasms', 'Diseases of the circulatory system', 'Diseases of the nervous system', 'Diseases of the respiratory system',
                 'Mental and behavioural disorders', 'Diseases of the skin and subcutaneous tissue' ] 
    
# selecting rows based on condition 
BE_subset_df = df_MortalityBE[df_MortalityBE['COD'].isin(values_to_keep)]

In [39]:
#df_MortalityBE.shape  #   41278 rows
#BE_subset_df.shape     #   17234 rows
#BE_subset_df.info()

(17234, 7)

In [40]:
BE_subset_df['COD'].value_counts()

Neoplasms                                       3628
Diseases of the circulatory system              3380
Diseases of the nervous system                  3196
Diseases of the respiratory system              3037
Mental and behavioural disorders                2817
Diseases of the skin and subcutaneous tissue    1176
Name: COD, dtype: int64

# Now it needs to be;

- added per quarter (sum per 3 month and sex, but within type and year)..
- multiplied by percentage (fraction) to make it correspond to NL death types..


In [58]:
!pip install -U pandasql

  Stored in directory: /Users/steven/Library/Caches/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [61]:
from pandasql import sqldf

In [81]:
q = "SELECT region, year, case when month IN (1,2,3) then 'Q1' when month IN (4, 5, 6) then 'Q2' when month IN (7,8,9) then 'Q3' when month in (10, 11, 12) then 'Q4' ELSE 'unknown' END as quarters, cod, sum(count) as deathcount FROM BE_subset_df GROUP BY REGION, YEAR, quarters, COD" 
newSQLtoPanda_df = sqldf(q, globals())

In [82]:
newSQLtoPanda_df.head(10)

,REGION,YEAR,quarters,COD,deathcount
0,2000,2009,Q1,Diseases of the circulatory system,5498
1,2000,2009,Q1,Diseases of the nervous system,622
2,2000,2009,Q1,Diseases of the respiratory system,2361
3,2000,2009,Q1,Diseases of the skin and subcutaneous tissue,37
4,2000,2009,Q1,Mental and behavioural disorders,709
5,2000,2009,Q1,Neoplasms,3981
6,2000,2009,Q2,Diseases of the circulatory system,4419
7,2000,2009,Q2,Diseases of the nervous system,491
8,2000,2009,Q2,Diseases of the respiratory system,1355
9,2000,2009,Q2,Diseases of the skin and subcutaneous tissue,26


&emsp;






# Do the same with the NL dataset.
import, and keep onl relevant columns

In [57]:
df_MortalityNL = pd.read_csv(GitURL_NetherlandsDeath, skiprows=3)

In [42]:
df_MortalityNL.head()
#I cant really get this correct, I can do so easily within the xls (by hand I mean) and the reupload and reimport.
# But not sure if for the assignment it is not better to do this within python..

,";""Totaal alle overledenen"";""Nieuwvormingen|Totaal nieuwvormingen"";""Nieuwvormingen|Kwaadaardige nv. van luchtpijp en long"";""Nieuwvormingen|Kwaadaardige nieuwvorming van borst"";""Nieuwvormingen|Overige nieuwvormingen"";""Psychische stoornis. en zktn zenuwstel.."";""Ziekten van hart en vaatstelsel|Totaal ziekten van hart en vaatstelsel"";""Ziekten van hart en vaatstelsel|Acuut hartinfarct"";""Ziekten van hart en vaatstelsel|Hersenvaatletsels"";""Ziekten van hart en vaatstelsel|Overige ziekten van hart en vaatstelsel"";""Ziekten van de ademhalingsorganen"";""Overige doodsoorzaken"""
0,"Perioden;""aantal"";""aantal"";""aantal"";""aantal"";""..."
1,"1996 1e kwartaal;""38552"";""9609"";""2073"";""870"";""..."
2,"1996 2e kwartaal;""33618"";""9426"";""2171"";""917"";""..."
3,"1996 3e kwartaal;""31499"";""9494"";""2109"";""883"";""..."
4,"1996 4e kwartaal;""33892"";""9742"";""2218"";""907"";""..."


In [45]:
#df.columns = ['Perioden', 'Totaal alle overledenen', 'Nieuwvormingen|Totaal nieuwvormingen', 'new_col4', etc..]

In [27]:
"""
Classificatie Nederland

Kwaadaardige nv. van luchtpijp en long 
aantal:  neoplasme * % luchtpijp en long

Nieuwvormingen|Kwaadaardige nieuwvorming van borst  Neoplasms
aantal: neoplasme * % kwaadaardig borst

Nieuwvormingen|Overige nieuwvormingen
aantal: neoplasme * % overig

Psychische stoornis. en zktn zenuwstel..
aantal: Mental and behavioural disorders  + Diseases of the nervous system                                                                    

Ziekten van hart en vaatstelsel|Totaal ziekten van hart en vaatstelsel
aantal:  Diseases of the circulatory system x % hart en vaat..

Ziekten van hart en vaatstelsel|Acuut hartinfarct
aantal: Diseases of the circulatory system x % heart attack

Ziekten van hart en vaatstelsel|Hersenvaatletsels
aantal: Diseases of the circulatory system x % brain circulatory etc

Ziekten van hart en vaatstelsel|Overige ziekten van hart en vaatstelsel
aantal: Diseases of the circulatory system x % other

Ziekten van de ademhalingsorganen
aantal: Diseases of the respiratory system 

Overige doodsoorzaken : Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified                3536

"""

'\nClassificatie Nederland\nKwaadaardige nv. van luchtpijp en long\naantal\n\nNieuwvormingen|Kwaadaardige nieuwvorming van borst  Neoplasms\naantal\n\nNieuwvormingen|Overige nieuwvormingen\naantal\n\nPsychische stoornis. en zktn zenuwstel..\naantal: Mental and behavioural disorders  + Diseases of the nervous system                                                                    \n\nZiekten van hart en vaatstelsel|Totaal ziekten van hart en vaatstelsel\naantal\n\nZiekten van hart en vaatstelsel|Acuut hartinfarct\naantal\n\nZiekten van hart en vaatstelsel|Hersenvaatletsels\naantal\n\nZiekten van hart en vaatstelsel|Overige ziekten van hart en vaatstelsel\naantal\n\nZiekten van de ademhalingsorganen\naantal\n\nOverige doodsoorzaken : Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified                3536\n\naantal\n'

In [ ]:
'''
heatwave health effects which lead to death: respiratory, cardiovascular | extra: skincancer, nervous system & mental..

- respiratory + cardiovascular are in the dataset itself
- skincancer voor belgie is in de dataset onder Diseases of the skin and subcutaneous tissue                                                           1176
- voor Nederland, sommeren we alle drie kanker kolommen en op basis van statistiek nemen we verhouding van dit totaal
- mental/psychological and nervous system ook in beide sets.


'''

In [54]:
# for now moving on with new dataset (newly uploaded, same dataset but row headers fixed manually)
df_MortalityNL = pd.read_excel(GitURL_NetherlandsDeath3)

In [55]:
df_MortalityNL.shape

(93, 13)

In [56]:
df_MortalityNL.head()
#had to change to xlsx and reupload to get the columns working properly
#will continue next time

,Perioden,Totaal alle overledenen,Nieuwvormingen|Totaal nieuwvormingen,Nieuwvormingen|Kwaadaardige nv. van luchtpijp en long,Nieuwvormingen|Kwaadaardige nieuwvorming van borst,Nieuwvormingen|Overige nieuwvormingen,Psychische stoornis. en zktn zenuwstel,Ziekten van hart en vaatstelsel|Totaal ziekten van hart en vaatstelsel,Ziekten van hart en vaatstelsel|Acuut hartinfarct,Ziekten van hart en vaatstelsel|Hersenvaatletsels,Ziekten van hart en vaatstelsel|Overige ziekten van hart en vaatstelsel,Ziekten van de ademhalingsorganen,Overige doodsoorzaken
0,1996 1e kwartaal,38552.0,9609.0,2073.0,870.0,6666.0,2036.0,14751.0,4414.0,3454.0,6883.0,4556.0,7600.0
1,1996 2e kwartaal,33618.0,9426.0,2171.0,917.0,6338.0,1772.0,12507.0,3673.0,3063.0,5771.0,3089.0,6824.0
2,1996 3e kwartaal,31499.0,9494.0,2109.0,883.0,6502.0,1619.0,11281.0,3385.0,2663.0,5233.0,2484.0,6621.0
3,1996 4e kwartaal,33892.0,9742.0,2218.0,907.0,6617.0,1680.0,12774.0,3786.0,3052.0,5936.0,2926.0,6770.0
4,1997 1e kwartaal,37134.0,9612.0,2203.0,926.0,6483.0,1976.0,13789.0,4015.0,3248.0,6526.0,4527.0,7230.0


In [53]:
# does not work but perhaps not needed anyway

#first make it easier by creating a df with only the relevant rows (Netherlands dataset)
#columns_to_keep = ['Perioden', 'Totaal alle overledenen', 'Nieuwvormingen|Totaal nieuwvormingen', 'Nieuwvormingen|Kwaadaardige nv. van luchtpijp en long',
#                 'Nieuwvormingen|Kwaadaardige nieuwvorming van borst', 'Nieuwvormingen|Overige nieuwvormingen','Psychische stoornis. en zktn zenuwstel', 'Ziekten van hart en vaatstelsel|Totaal ziekten van hart en vaatstelsel',
#                  'Ziekten van hart en vaatstelsel|Acuut hartinfarct', 'Ziekten van hart en vaatstelsel|Hersenvaatletsels', 'Ziekten van hart en vaatstelsel|Overige ziekten van hart en vaatstelsel', 'Ziekten van de ademhalingsorganen', 'Overige doodsoorzaken'] 
    
# selecting rows based on condition 
#NL_subset_df = df_MortalityNL[[df_MortalityNL.isin(columns_to_keep)]